In [1]:
import torch
import os.path as osp
from utils import *
from args import *
from dataset import *
from tqdm import tqdm
from torch_geometric.loader import DataLoader

/home/catcolia/anaconda3/envs/material/lib/python3.12/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/catcolia/anaconda3/envs/material/lib/python3.12/site-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/home/catcolia/anaconda3/envs/material/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hypo_dataset = HypoDataset(args)

batch_size = 1000
dataset = hypo_dataset[0]
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


def load_model(model_path, file_name="checkpoint.pt", load_dict=False, map_location=get_device()):
    model_path = osp.join(model_path, file_name)
    data = torch.load(model_path, map_location=map_location)
    if load_dict is True:
        data["model"].load_state_dict(data["model_state_dict"])
    model = data["model"]
    return model, data


@torch.no_grad()
def predict(model, data, batch_size, device):
    model.eval()

    pbar = tqdm(total=(batch_size + 1))
    pbar.set_description("Predicting")

    res_out = torch.zeros(0).to(device)
    for i, d in enumerate(data):
        d = d.to(device)
        out = model(d)
        res_out = torch.cat((res_out, out), 0)
        pbar.update(1)
    pbar.close()
    return res_out

Processing...
hypothesis compounds dataset processing:   0%|          | 120/1254570 [00:00<20:44, 1008.36it/s]

save points:  [4180, 8361, 12542, 16723, 20904, 25085, 29266, 33447, 37628, 41818]


hypothesis compounds dataset processing:  34%|███▍      | 432060/1254570 [14:33<18:10, 753.97it/s]  

: 

In [3]:
model_path = osp.join(args["result_path"], "CEAL/1715083645273902")
model, _ = load_model(model_path, file_name="checkpoint.pt", load_dict=False)

In [4]:
device = get_device()
out = predict(model, dataloader, batch_size, device)

Predicting:   3%|▎         | 34/1001 [01:59<58:01,  3.60s/it]  

KeyboardInterrupt: 

------------

In [12]:
print(dataloader)
print(len(dataset))

125430


In [4]:
hypo_dataset[9][0]

Data(edge_index=[2, 56], edge_attr=[56, 1], x=[8, 84], id=1128871)

In [8]:
for i in dataloader:
    print(i.id)

tensor([     1,      2,      3,  ..., 125428, 125429, 125430])


In [20]:
total = 0
for idx, data in enumerate(hypo_dataset):
    total += len(data)
    print(len(data))
    # print(idx)
    # begin = data[0].id - 30
    # for i, d in enumerate(data):
    #     d.id = i + begin
    # torch.save(data, osp.join(args["hypothesis_dataset"]["data_dir"], f"{args['hypothesis_dataset']['data_filename']}_{idx+1}.pt"))

125430
125430
125430
125430
125430
125430
125430
125430
125430
125700
